In [1]:
import os, pygame, random, math
import numpy as np
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.3, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
colours = {"cyan": (0, 255, 255, 255),
           "blue": (0, 0, 255, 255),
           "orange": (255, 165, 0, 255),
           "yellow": (255, 255, 0, 255),
           "green": (0, 255, 0, 255),
           "purple": (160, 32, 240, 255),
           "red": (255, 0, 0, 255),
           "green2": (165, 255, 81),
            "red2": (255, 76, 76),
            "grey": (140, 140, 140),
            "black": (0, 0, 0),
            "white": (255, 255, 255)}

# Deterministic Pathfinder

In [7]:
class DeterministicPathfinder():
    def __init__(self, i, j, cellsize, type="det"):
        os.environ['SDL_VIDEO_CENTERED'] = '1'
        # keep running
        self.game = True
        # store values
        self.i_count = i
        self.j_count = j
        self.cell_size = cellsize
        # setup pygame
        pygame.init()
        self.window = pygame.display.set_mode((self.j_count * self.cell_size, self.i_count * self.cell_size))
        pygame.display.set_caption("Maze")
        #pygame.display.set_icon(pygame.image.load("icon.png"))
        self.clock = pygame.time.Clock()
        # initialise values 
        self.path = []
        self.visited = []
        self.grid = np.zeros((self.i_count, self.j_count))
        self.start = (0, 0)
        self.end = (0, 0)
        self.type = type
        # colour codes
        self.ccodes = {0: "white", 1: "black", 2: "blue", 3: "green", 4: "red", 5: "grey"}

    def create_grid(self):
        # random black squres
        for i in range(self.i_count):
            for j in range(self.j_count):
                u = np.random.uniform(0,1)
                if u < 0.2:
                    self.grid[i,j] = 1
        
        # random start and end: ensure not equal
        while self.start == self.end:
            self.start = (random.randint(0, self.i_count - 1), random.randint(0 , self.j_count - 1))
            self.end = (random.randint(0, self.i_count - 1), random.randint(0 , self.j_count - 1))
        
        # colour grid
        self.grid[self.start] = 2
        self.grid[self.end] = 3

        # add start to path and visited squares
        self.path.append(self.start)
        self.visited.append(self.start)

    def render(self):
        # define square function
        def square(colour, i, j):
            pygame.draw.rect(self.window, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))

        # draw tiles
        for i in range(self.i_count):
            for j in range(self.j_count):
                colour = colours[self.ccodes[int(self.grid[i,j])]]
                square(colour, i, j)
        pygame.display.update()

    def update(self):
        """Take grid status and perform a move."""
        # moved checker
        moved = False
        # current position
        current_pos = self.path[-1]
        i, j = current_pos
        # 4 cardinal neighbours
        if self.type == "det":
            # deterministic direction selection
            direction = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        elif self.type == "random":
            # random direction selection
            direction = random.sample([(-1, 0), (0, 1), (1, 0), (0, -1)], k=4)
        for di, dj in direction:
            if i + di < 0 or i + di >= self.i_count or j + dj < 0 or j + dj >= self.j_count:
                # outside of grid
                pass
            else:
                # inside of grid
                pos = (i + di, j + dj)

                # neighbour is end
                if pos == self.end:
                    print("Found")
                    self.game = False
                    moved = True
                    break

                # unvisited square
                elif int(self.grid[pos]) == 0:
                    # move there
                    self.path.append(pos)
                    self.visited.append(pos)
                    # colour: current red, new blue
                    self.grid[current_pos] = 4
                    self.grid[pos] = 2
                    # moved 
                    moved = True
                    break
        
        # if not moved: no unvisited spaces, dead end
        if not moved:
            try:
                # backtrack once along path
                dead_pos = self.path.pop()
                # colour: dead end grey, backstep blue
                self.grid[dead_pos] = 5
                self.grid[self.path[-1]] = 2
            except IndexError:
                # nowhere to backtrack to: impossible
                print("Not possible")
                self.game = False


    def user_input(self):
        """Take game status and process input."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.game = False
                break
            # listen for key presses
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.game = False
                    break

    def run(self):
        # create grid
        self.create_grid()
        # draw initial state
        self.render()
        while self.game:
            # input (only to quit)
            self.user_input()
            # wait to listen for inputs
            self.clock.tick(0)
            # update
            self.update()
            # draw
            self.render()
        pygame.quit()

In [11]:
dpt = DeterministicPathfinder(100, 100, 5, "random")
dpt.run()